# Кластеризация чеков
Автор: _Хасанов Расим 11-804_

#### Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

#### Загрузка данных

In [2]:
DATA_URL = 'https://github.com/ydooG/receipts_clustering/tree/master/core/static/core/data/receipts.xlsx'
raw_data = pd.read_excel(DATA_URL)
raw_data.sample(n=10)

,Unnamed: 0,id,protocolVersion,operationType,cashTotalSum,shiftNumber,counterSubmissionSum,totalSum,ecashTotalSum,nds18,...,items.nds10,items.nds18,items.paymentAgentByProductType,items.modifiers,userProperty.key,userProperty.value,propertiesUser.propertyValue,propertiesUser.propertyName,propertiesUser.key,propertiesUser.value
208,208,778609825,2.0,1,2500,70,0.0,2500,0,NaN,...,NaN,NaN,NaN,NaN,АВТОБУС (городской маршрут),№ транспортного средства 682,NaN,NaN,NaN,NaN
682,682,666903829,NaN,1,0,25,NaN,10000,10000,1667.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,659,666903100,NaN,1,3000,10,NaN,3000,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,256,778609131,NaN,1,0,30,NaN,30756,30756,5126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,1000,666908541,2.0,1,0,110,0.0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,294,778609898,NaN,1,0,14,NaN,20000,20000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,928,666906366,NaN,1,2300,54,NaN,2300,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,1084,666908914,2.0,1,14600,110,0.0,14600,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,79,778608319,NaN,1,0,19,NaN,1350,1350,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,188,778607956,2.0,1,0,147,0.0,37310,37310,6218.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Ручной анализ датасета
Так как задача состоит в кластеризации наименований товаров, то стоит рассматривать только названия товаров. Цену товара тоже не имеет особого смысла, так как: "Картофель за 50р/кг", тот же самый, что и "Картофель за 70р/кг".<br><br>
Таким образом задача сводится к области NLP. То есть: определить похожесть/различность слов

In [6]:
df = raw_data[['items.name']]
